# Python Binding
## Henry Schreiner
## PyCHEP 2018


# Focus

* What Python bindings do
* What tools are available
* How Python bindings work

# Caviats

* Will cover C++ and C binding only
* Will not cover every tool available
* Will not cover `cppyy` in detail (but see next talk)
* Python 2 is dying, long live Python 3!

# Selection:
* C Binding: ctypes, <pypy based thing>
* Manual binding (quick look)
* SWIG (Too automatic)
* Cython (Two languages)
* PyBind11 (just right)
* Special mention: Numba, cppyy

# Preperation

This talk is interactive, and uses `cookiecutter` to set up code for you to play with. Install CookieCutter:

```shell
pip install cookiecutter
```

Then run:

```shell
cookiecutter gh:henryiii/pybindings_cc
```

Answer the questions, and then you'll have a new directory ready to go!

# Bindngs: What is it?

Bindings allow a function(alitiy) to be accessed from Python.

In [1]:
%%writefile simple.c

float square(float x) {
    return x*x;
}

Overwriting simple.c


### Desired usage in Python:
```python
y = square(x)
```

C bindings are very easy. Just compile into a shared library, then open it in python with the built in `ctypes` module:

In [2]:
!cc simple.c -shared -o simple.so

In [3]:
from ctypes import cdll, c_float
lib = cdll.LoadLibrary('simple.so')
lib.square.argtypes = (c_float,)
lib.square.restype = c_float
lib.square(2.0) == 4.0

True

See [AmpGen](https://gitlab.cern.ch/lhcb/Gauss/blob/LHCBGAUSS-1058.AmpGenDev/Gen/AmpGen/options/ampgen.py) Python interface. In Pythonista for iOS, we can even use `ctypes` to access Apple's public APIs!

# Why do we need more?

For some cases, this is enougth. But often, you have an elegant (hopefully) C++ api. You can't reduce this down to `export "C"`. You'd like to be able to do the same thing in Python that you'd do in C++. You need overloading, classes, memory managment, etc.

Let's start looking at ways to bind C++, but let's try one more C example first:

In [4]:
%%writefile pysimple.c

#include <Python.h>

float square(float x) {
    return x*x; }

/* C reminder: static means only visible in this file (not exported) */
static PyObject* square_wrapper(PyObject* self, PyObject* args) {
  float input, result;

  if (!PyArg_ParseTuple(args, "f", &input)) {
      return NULL; }

  result = square(input);
  return PyFloat_FromDouble(result);}

static PyMethodDef pysimple_methods[] = {
 { "square", square_wrapper, METH_VARARGS, "Square function" },
 { NULL, NULL, 0, NULL } };

#if PY_MAJOR_VERSION >= 3
static struct PyModuleDef pysimple_module = {
    PyModuleDef_HEAD_INIT, "pysimple", NULL, -1, pysimple_methods};

PyMODINIT_FUNC PyInit_pysimple(void) {
    return PyModule_Create(&pysimple_module);
}
#else
DL_EXPORT(void) initpysimple(void) {
  Py_InitModule("pysimple", pysimple_methods); }
#endif

Overwriting pysimple.c


### Build:

In [5]:
from sysconfig import get_paths
inc = get_paths()['include']

In [6]:
!cc -I {inc} -shared -o pysimple.so pysimple.c -undefined dynamic_lookup

### Run:

In [7]:
import pysimple
pysimple.square(2.0) == 4.0

True

# SWIG : automated bindins

* Produces "automatic" bindings
* Works with many output languages
* Has supporting module built into CMake

Downsides:

* Can be all or nothing
* Hard to customize
* Customizations tend to be language specific
* Slow development

In [8]:
%%writefile SimpleClass.hpp
#pragma once

class Simple {
    int x;
  public:
    Simple(int x):x(x) {}
    int get() const {return x;}
};

Overwriting SimpleClass.hpp


In [9]:
%%writefile SimpleSWIG.i

%module simpleswig
%{
/* Includes the header in the wrapper code */
#include "SimpleClass.hpp"
%}
 
/* Parse the header file to generate wrappers */
%include "SimpleClass.hpp"

Overwriting SimpleSWIG.i


In [10]:
!swig -python -c++ SimpleSWIG.i

In [11]:
!c++ -shared SimpleSWIG_wrap.cxx -I {inc} -o _simpleswig.so -undefined dynamic_lookup

In [12]:
import simpleswig
x = simpleswig.Simple(2)
x.get()

2

# Cython

* Built to be a Python+C language for high performance compututations
* Performance computation space in competition with numba
* Due to design, also makes binding easy

Downsides:
* Requires learning a new(ish) language
* Have to think with three hats
* Very verbose

# Speed comparison

In [13]:
def f(x):
    for _ in range(100000000):
        x=x+1
    return x

In [14]:
%%timeit
f(1)

6.11 s ± 180 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
%load_ext Cython

In [16]:
%%cython

def f(int x):
    for _ in range(100000000):
        x=x+1
    return x

In [17]:
%%timeit
f(23)

65.2 ns ± 1.16 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [18]:
import numba
@numba.jit
def f(x):
    for _ in range(100000000):
        x=x+1
    return x

In [19]:
%time
f(41)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 6.2 µs


100000041

In [20]:
%%timeit
f(41)

200 ns ± 14.1 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


# Binding with Cython

In [21]:
%%writefile simpleclass.pxd
# distutils: language = c++

cdef extern from "SimpleClass.hpp":
    cdef cppclass Simple:
        Simple(int x)
        int get()

Overwriting simpleclass.pxd


In [22]:
%%writefile cythonclass.pyx
# distutils: language = c++

from simpleclass cimport Simple as cSimple

cdef class Simple:
    cdef cSimple *cself
    
    def __cinit__(self, int x):
        self.cself = new cSimple(x)
    
    def get(self):
        return self.cself.get()
    
    def __dealloc__(self):
        del self.cself

Overwriting cythonclass.pyx


In [23]:
!cythonize cythonclass.pyx

Compiling /Users/henryiii/git/presentations/physics/pychep/cythonclass.pyx because it changed.
[1/1] Cythonizing /Users/henryiii/git/presentations/physics/pychep/cythonclass.pyx


In [24]:
!g++ cythonclass.cpp -shared -I {inc} -o cythonclass.so -undefined dynamic_lookup

In [25]:
import cythonclass
x = cythonclass.Simple(3)
x.get()

3

# PyBind11

* Designed for making bindings
* Pure C++11 (no new language required)
* Builds remain simple and don't require preprocessing
* Easy to customize
* Great Gitter community
* Similar to Boost::Python, but easier to build

Downsides:
* Still verbose
* Development variable
* Requires C++11 (and better with C++14)

In [26]:
%%writefile pybindclass.cpp

#include <pybind11/pybind11.h>
#include "SimpleClass.hpp"

namespace py = pybind11;

PYBIND11_MODULE(pybindclass, m) {
    py::class_<Simple>(m, "Simple")
        .def(py::init<int>())
        .def("get", &Simple::get)
    ;
}

Overwriting pybindclass.cpp


In [27]:
!c++ -std=c++11 pybindclass.cpp -shared -I {inc} -o pybindclass.so -undefined dynamic_lookup

In [28]:
import pybindclass
x = pybindclass.Simple(4)
x.get()

4

# Cppyy

* Born from ROOT bindings
* Built on top of Cling
* JIT, so can handle templates

Downsides:

* Header code runs in Cling
* Can be slow (PyPy supposed to be faster)

In [29]:
import cppyy

In [30]:
cppyy.include('SimpleClass.hpp')

In [31]:
x = cppyy.gbl.Simple(5)
x.get()

5